# build an agent 构建代理

In [1]:
# Import relevant functionality
from langchain.chat_models import init_chat_model
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = MemorySaver()
# model = init_chat_model("anthropic:claude-3-5-sonnet-latest")
model = init_chat_model("gpt-4o-mini", model_provider="openai")
search = TavilySearch(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [2]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

input_message = {
    "role": "user",
    "content": "Hi, I'm Bob and I live in SF.",
}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob and I live in SF.
================================== Ai Message ==================================

Hi Bob! It's great to meet you. How can I assist you today?


In [3]:
input_message = {
    "role": "user",
    "content": "What's the weather where I live?",
}

for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's the weather where I live?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_72DOwhRD4YyuXYKBmMYITsod)
 Call ID: call_72DOwhRD4YyuXYKBmMYITsod
  Args:
    query: current weather San Francisco
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather San Francisco", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in San Francisco", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1756651789, 'localtime': '2025-08-31 07:49'}, 'current': {'last_updated_epoch': 1756651500, 'last_updated': '2025-08-31 07:45', 'temp_c': 15.6, 't

### 定义工具

In [4]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("What is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': 'What is the weather in SF', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1756652271, 'localtime': '2025-08-31 07:57'}, 'current': {'last_updated_epoch': 1756651500, 'last_updated': '2025-08-31 07:45', 'temp_c': 15.6, 'temp_f': 60.1, 'is_day': 1, 'condition': {'text': 'Fog', 'icon': '//cdn.weatherapi.com/weather/64x64/day/248.png', 'code': 1135}, 'wind_mph': 2.5, 'wind_kph': 4.0, 'wind_degree': 246, 'wind_dir': 'WSW', 'pressure_mb': 1014.0, 'pressure_in': 29.94, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 80, 'cloud': 25, 'feelslike_c': 15.6, 'feelslike_f': 60.1, 'windchill_c': 12.6, 'windchill_f': 54.7, 'heatindex_c': 12.9, 'heatindex_f': 55.2, 'dewpoint_c': 12.7, 'dewpoint_f'

In [5]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4.1", model_provider="openai")

In [6]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hello! How can I help you today?'

In [7]:
model_with_tools = model.bind_tools(tools)

query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hello! How can I help you today? 😊

Tool calls: []


In [8]:
query = "Search for the weather in ShenZhen"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'current weather in Shenzhen', 'search_depth': 'basic'}, 'id': 'call_GZMmXJKV0lsEaQnOyX44vmJO', 'type': 'tool_call'}]


In [11]:
from langgraph.prebuilt import create_react_agent

# 创建代理
agent_executor = create_react_agent(model, tools)

# 运行代理
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

# 调用工具
input_message = {"role": "user", "content": "Search for the weather in shenzhen"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hello! How can I help you today? 😊
================================ Human Message =================================

Search for the weather in shenzhen
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_zrREbEuimifanT2jFZMrnA69)
 Call ID: call_zrREbEuimifanT2jFZMrnA69
  Args:
    query: current weather in Shenzhen
    search_depth: basic
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather in Shenzhen", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Shenzhen", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Shenzhen', 'region': 'Guangdong', 'country': 'China', 'lat': 22.5333, 'lon': 114.1333, 'tz_id': 'Asia/Shangh

In [12]:
# 流式消息
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Search for the weather in shenzhen
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_UYX5jD6qdpt9cjC1IKJkeisi)
 Call ID: call_UYX5jD6qdpt9cjC1IKJkeisi
  Args:
    query: current weather in Shenzhen
    search_depth: basic
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather in Shenzhen", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Shenzhen", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Shenzhen', 'region': 'Guangdong', 'country': 'China', 'lat': 22.5333, 'lon': 114.1333, 'tz_id': 'Asia/Shanghai', 'localtime_epoch': 1756652407, 'localtime': '2025-08-31 23:00'}, 'current': {'last_updated_epoch': 1756652400, 'last_updated': '2025-08-31 23:00', 'temp_c': 29.3, 'temp_f': 84.7, 'is_day': 0, 'condi

In [13]:
for step, metadata in agent_executor.stream(
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

The| current| weather| in| Shenzhen|:

|-| Temperature|:| |29|.|3|°C| (|84|.|7|°F|)
|-| Condition|:| Clear|
|-| Hum|idity|:| |79|%
|-| Wind|:| |6|.|5| km|/h| (|E|SE| direction|)
|-| Fe|els| like|:| |34|°C| (|93|.|2|°F|)
|-| Pressure|:| |100|9| mb|
|-| Visibility|:| |10| km|

|Source|:| Weather|API|

|If| you| need| more| details| or| a| forecast|,| let| me| know|!|

In [14]:
# 添加到内存
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

input_message = {"role": "user", "content": "Hi, I'm Bob!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob!
================================== Ai Message ==================================

Hi Bob! 👋 How can I help you today?


In [15]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is Bob!


In [16]:
config = {"configurable": {"thread_id": "xyz123"}}

input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

You haven’t told me your name yet. If you’d like me to remember it or use it in our conversation, please let me know!
